<a href="https://colab.research.google.com/github/manhhad32/nlp/blob/master/exPhoBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dưới đây là cách tải và sử dụng PhoBERT cho một tác vụ như phân loại văn bản:

In [13]:
from transformers import AutoModel, AutoTokenizer

# Tải mô hình PhoBERT và tokenizer
model = AutoModel.from_pretrained("vinai/phobert-base")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

# Mẫu văn bản tiếng Việt
text = "PhoBERT là một mô hình mạnh mẽ cho tiếng Việt."

# Tokenize văn bản
inputs = tokenizer(text, return_tensors="pt")

# Tạo embedding từ mô hình PhoBERT
outputs = model(**inputs)

# Đầu ra của PhoBERT
last_hidden_states = outputs.last_hidden_state
print(last_hidden_states)


tensor([[[ 0.1177,  0.3734, -0.7098,  ..., -0.3446,  0.2745, -0.1793],
         [ 0.1052,  0.4616, -0.4968,  ..., -0.5583,  0.0744,  0.0508],
         [ 0.0632,  0.6439, -0.2283,  ..., -0.4015,  0.1891, -0.0091],
         ...,
         [ 0.2425,  0.5334, -0.2021,  ...,  0.0661,  0.1878, -0.0437],
         [ 0.1909,  0.5635,  0.0697,  ...,  0.1532,  0.1026, -0.1234],
         [ 0.0503,  0.3339, -0.5908,  ..., -0.2530,  0.0211, -0.2728]]],
       grad_fn=<NativeLayerNormBackward0>)


Chuẩn Bị Dữ Liệu
Giả sử chúng ta có một tập dữ liệu phân loại văn bản với các nhãn "positive" và "negative".

In [14]:
# Mẫu dữ liệu
texts = ["Tôi rất thích sản phẩm này!", "Sản phẩm này thật tệ."]
labels = [1, 0]  # 1: positive, 0: negative


Tokenize Dữ Liệu

In [15]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Tạo DataLoader

In [16]:
import torch
from torch.utils.data import DataLoader, TensorDataset

labels = torch.tensor(labels)
dataset = TensorDataset(inputs['input_ids'], inputs['attention_mask'], labels)
dataloader = DataLoader(dataset, batch_size=2)


Xây Dựng Mô Hình

In [17]:
import torch.nn as nn
from transformers import AutoModel

class PhoBERTClassifier(nn.Module):
    def __init__(self):
        super(PhoBERTClassifier, self).__init__()
        self.phobert = AutoModel.from_pretrained("vinai/phobert-base")
        self.classifier = nn.Linear(768, 2)  # PhoBERT base có đầu ra 768 chiều

    def forward(self, input_ids, attention_mask):
        outputs = self.phobert(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs.last_hidden_state[:, 0, :]  # Lấy embedding của token [CLS]
        logits = self.classifier(cls_output)
        return logits

model = PhoBERTClassifier()


Huấn Luyện Mô Hình

In [20]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=1e-5)

# Vòng lặp huấn luyện đơn giản
for epoch in range(3):
    for batch in dataloader:
        input_ids, attention_mask, labels = batch
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = nn.CrossEntropyLoss()(outputs, labels)
        loss.backward()
        optimizer.step()
        print(f"Loss: {loss.item()}")


Loss: 0.1432359516620636
Loss: 0.1121024340391159
Loss: 0.07950952649116516
